In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%run -i '/content/drive/MyDrive/Colab Notebooks/Living Cell imaging/C2CGNN/Preprocessing/image processing/GMD.py'
%run -i '/content/drive/MyDrive/Colab Notebooks/Living Cell imaging/C2CGNN/Preprocessing/image processing/naming.ipynb'
%run -i '/content/drive/MyDrive/Colab Notebooks/Living Cell imaging/C2CGNN/Preprocessing/image processing/iq_getspectralpar.ipynb'
%run -i '/content/drive/MyDrive/Colab Notebooks/Living Cell imaging/C2CGNN/Preprocessing/image processing/iq_getcameraqe.ipynb'
%run -i '/content/drive/MyDrive/Colab Notebooks/Living Cell imaging/C2CGNN/Preprocessing/image processing/iq_getlightsourcespectra.ipynb'

FileNotFoundError: ignored

FileNotFoundError: ignored

In [ ]:
import numpy as np
import re

In [ ]:
def iman_powerratio_est(MD, SP=None, cn=None):

    # Default Fret pair = CFP/YFP
    if cn is None or not cn:
        cn = ['cfp', 'yfp']
    #Get matching channel IDs
    id = [i for i, channel in enumerate(MD["exp"]["Channel"]) for name in cn if re.search(name, channel, re.IGNORECASE)]

    #Remove irrelevant channels
    fnn = ['Channel', 'Exposure', 'ExVolt', 'Filter', 'FPhore', 'ExLine', 'ExWL']
    for name in fnn:
        if name in MD["exp"]:
            MD["exp"][name] = [MD["exp"][name][i] for i in id]

    #Ensure spectral parameters are available
    if SP is None or not SP:
        SP = iq_getspectralpar()

    #Assert proper names for MetaData on FPhores and Filters
    MD["exp"]["FPhore"], MD["exp"]["Filter"] = iman_naming('match', MD["exp"]["FPhore"], MD["exp"]["Filter"])

    #Get Light source spectral power
    lsPower, cc = iq_getlightsourcespectra(MD)
    lsn = [name for name in lsPower if name != 'WaveLength']

    #If Multi Line source, assemble line list
    is_multiline = False
    if "Line" in lsPower[lsn[0]] and "ExLine" in MD["exp"] and MD["exp"]["ExLine"]:
        is_multiline = True
    #Double check MetaData format
        if not isinstance(MD["exp"]["ExVolt"], list):
            MD["exp"]["ExVolt"] = [MD["exp"]["ExVolt"]]
    else:
        # If single line, ensure numeric array MD
        if isinstance(MD["exp"]["ExVolt"], list):
            MD["exp"]["ExVolt"] = np.array(MD["exp"]["ExVolt"])
        if isinstance(MD["exp"]["Exposure"], list):
            MD["exp"]["Exposure"] = np.array(MD["exp"]["Exposure"])
    #Get camera QE (quantum efficiency) curve
    QEcam, cdb,db = iq_getcameraqe(MD["cam"]["Desc"], SP["WaveLength"])
    #Evaluate channel data to get intensity gains
    rig = [0, 0]
    for s in range(2):
        # IF Multi-line, use the calibrated channel (from getlightsourcespectra)
        if is_multiline:
            lsp = cc[s]
        else:
            # IF Single-line, apply cal factor
            lsp = np.multiply(lsPower[lsn[0]]["spec"], cc[s])
        # Define relevant filter and fluorophore spectra
        fp = SP[MD["exp"]["FPhore"][s]]
        ft = SP[MD["exp"]["Filter"][s]]

        #Calculate relative intensity gain
        rig[s] = (
            np.sum(lsp *ft["ex"][:,0]* fp["ab"][:,0]) *
            np.sum(ft["em"][:,0] * fp["em"][:,0] * QEcam) *
            fp["mec"] * fp["QY"] * MD["exp"]["Exposure"][s] /
            np.sum(fp["em"][:,0])
        )
    #Take ratio of RIG to get Power Ratio (e.g. Donor to Acceptor in FRET pair)
    prat = rig[0] / rig[1]

    return prat

# You would still need to define or translate other functions like:
# - iq_getspectralpar()
# - iman_naming()
# - iq_getlightsourcespectra()
# - iq_getcameraqe()


# Usage

MD = mdi
fc = ['cfp','yfp']
SP = []

prat = iman_powerratio_est(MD,SP,fc)

# Later use

if cn is None or not cn:
    cn = ['cfp', 'yfp']
#Get matching channel IDs
id = [i for i, channel in enumerate(MD["exp"]["Channel"]) for name in cn if re.search(name, channel, re.IGNORECASE)]

#Remove irrelevant channels
fnn = ['Channel', 'Exposure', 'ExVolt', 'Filter', 'FPhore', 'ExLine', 'ExWL']
for name in fnn:
    if name in MD["exp"]:
        MD["exp"][name] = [MD["exp"][name][i] for i in id]

#Ensure spectral parameters are available
if SP is None or not SP:
    SP = iq_getspectralpar()

#Assert proper names for MetaData on FPhores and Filters
MD["exp"]["FPhore"], MD["exp"]["Filter"] = iman_naming('match', MD["exp"]["FPhore"], MD["exp"]["Filter"])

#Get Light source spectral power
lsPower, cc = iq_getlightsourcespectra(MD)
lsn = [name for name in lsPower if name != 'WaveLength']

#If Multi Line source, assemble line list
is_multiline = False
if "Line" in lsPower[lsn[0]] and "ExLine" in MD["exp"] and MD["exp"]["ExLine"]:
    is_multiline = True
#Double check MetaData format
    if not isinstance(MD["exp"]["ExVolt"], list):
        MD["exp"]["ExVolt"] = [MD["exp"]["ExVolt"]]
else:
    # If single line, ensure numeric array MD
    if isinstance(MD["exp"]["ExVolt"], list):
        MD["exp"]["ExVolt"] = np.array(MD["exp"]["ExVolt"])
    if isinstance(MD["exp"]["Exposure"], list):
        MD["exp"]["Exposure"] = np.array(MD["exp"]["Exposure"])
#Get camera QE (quantum efficiency) curve
QEcam, cdb,db = iq_getcameraqe(MD["cam"]["Desc"], SP["WaveLength"])
#Evaluate channel data to get intensity gains
rig = [0, 0]
for s in range(2):
    # IF Multi-line, use the calibrated channel (from getlightsourcespectra)
    if is_multiline:
        lsp = cc[s]
    else:
        # IF Single-line, apply cal factor
        lsp = np.multiply(lsPower[lsn[0]]["spec"], cc[s])
    # Define relevant filter and fluorophore spectra
    fp = SP[MD["exp"]["FPhore"][s]]
    ft = SP[MD["exp"]["Filter"][s]]
    
    #Calculate relative intensity gain
    rig[s] = (
        np.sum(lsp *ft["ex"][:,0]* fp["ab"][:,0]) *
        np.sum(ft["em"][:,0] * fp["em"][:,0] * QEcam) *
        fp["mec"] * fp["QY"] * MD["exp"]["Exposure"][s] /
        np.sum(fp["em"][:,0])
    )
#Take ratio of RIG to get Power Ratio (e.g. Donor to Acceptor in FRET pair)
prat = rig[0] / rig[1]

array([4.440968396558762], dtype=object)